In [1]:
from gensim.models import FastText
from gensim.models.fasttext import load_facebook_vectors
from gensim.test.utils import datapath
from sklearn.cluster import KMeans 
import numpy as np
import matplotlib.pyplot as plt
import sklearn.cluster
import pandas as pd

# Load word embeddings (FastText model)

In [2]:
## specify path for embeddings file

# # these embeddings come with the gensim package
# cap_path = datapath("crime-and-punishment.bin")
cap_path = datapath("lee_fasttext.bin")

# # The following embeddings were downloaded from:
# # https://fasttext.cc/docs/en/pretrained-vectors.html
# # and saved to "data" directory.
# cap_path = "data/wiki/wiki.en.bin" 

## load the embeddings
fbkv = load_facebook_vectors(cap_path)
wind_terms = []
n_rows = 13
for i in np.arange(0,n_rows):
    rows_before = max(i, 0)
    rows_after = n_rows-(i+1)
    wind_terms.append(
        pd.read_csv("wind_terms_no_commas.txt", sep=", ",skiprows=rows_before, skipfooter = rows_after, header=None, engine="python")
    )

#categories = ["calm", "light air", "light breeze", "gentle breeze", "moderate breeze", "fresh breeze",
    #"strong breeze", "near gale", "gale", "strong gale", "storm", "violent storm", "hurricane"]

fbkv_list = []
for term in wind_terms:
    fbkv_list.append(fbkv[term])

fbkv_vector = []
indices = []
for i in range(n_rows):
    row_length = len(fbkv_list[i])
    indices.append(row_length)
    for j  in range(row_length):
        fbkv_vector.append(fbkv_list[i][j])
    #wind_array = np.asarray(fbkv_list, dtype="object")

wind_terms

# IMPERFECT SOLUTION TO THE WEIRD 3 DIMENSIONALITY OF WIND TERMS

#vector without the last category
wind_vector = []

for i in range(n_rows):
    for j in range(indices[i]):
        #print("loop iteration: " + str(i) + " column: " + str(j))
        wind_vector.append(wind_terms[i][j][0])

#number 10 is giving us trouble
#df = pd.DataFrame(wind_array) 

# Make training data by embedding the logbook data

In [3]:
X_train = fbkv_vector

# Train clustering model on embedded data (no seed)

In [4]:
beaufort_terms = [fbkv["calm"], fbkv["light air"], fbkv["light breeze"], fbkv["gentle breeze"], fbkv["moderate breeze"], fbkv["fresh breeze"], fbkv["strong breeze"], fbkv["near gale"], fbkv["gale"], fbkv["strong gale"], fbkv["storm"], fbkv["violent storm"], fbkv["hurricane"]]

kmeans_model = sklearn.cluster.KMeans(n_clusters=13, init=beaufort_terms, n_init="auto")

## fit the model to training data
kmeans_model.fit(X_train)
## cluster the training data
beaufort_predictions = kmeans_model.predict(X_train)

beaufort_predictions

array([ 1, 10,  0, ..., 10,  2,  9], dtype=int32)

# Print the cluster

In [5]:
#filter rows of original data
d = {'term': wind_vector, 'cluster': beaufort_predictions}
df = pd.DataFrame(data = d)


columns = {}
for i in range(n_rows):
    columns.update({i : df[beaufort_predictions == i]})
    
df = df.sort_values(by = 'cluster', axis = 0)

df.to_csv("new.txt", sep=' ', columns = ['term'], index = False, mode='a') # prints all wind terms


cluster_list = []

def is_str(variable):
    if isinstance(variable, str):
        return True
    else:
        return False
        
for i in range(n_rows):
    df_u = df[df['cluster'] == i]
    term_list = df_u['term'].tolist()
    cluster_list.append(filter(is_str, term_list))



string_list = []
for i in range(n_rows):
    string_list.append(', '.join(cluster_list[i]))

file = open('seeded_clusters.txt','w')
file.write("\nK-means clusters (Beaufort terms used as seeds)\n")
for i in range(n_rows):
	file.write("\nCluster " + str(i) + "\n" + string_list[i] + "\n")
file.close()
#df[beaufort_predictions == 0].to_csv('cluster0.txt', header=True, index=None, sep=' ', mode='a')